Here is the R8 R52 dataset on kaggle: https://www.kaggle.com/weipengfei/ohr8r52
here is an example of using Glove embeddings with the R8 dataset: https://www.kaggle.com/dvircohen0/text-classification-with-word-vectors
Here is where to get the Glove embeddings: https://www.kaggle.com/watts2/glove6b50dtxt
Here is the link to the paper I want to implement: Text Level Graph Neural Network for Text Classification (https://arxiv.org/pdf/1910.02356.pdf)
Here is a link to an implementation of the research paper with pytorch: https://github.com/Cynwell/Text-Level-GNN